In [1]:
# 5. Import the required modules

import networkx as nx
import nx_arangodb as nxadb

from arango import ArangoClient

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import re

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

import os
import networkx as nx
import nx_arangodb as nxadb

os.environ["DATABASE_HOST"] = "https://b61c3b83bfe6.arangodb.cloud:8529"
os.environ["DATABASE_USERNAME"] = "root"
os.environ["DATABASE_PASSWORD"] = "RHr0KzkRUVlp61IisH8G"
os.environ["DATABASE_NAME"] = "DAC_devops_log"

db = ArangoClient(hosts="https://b61c3b83bfe6.arangodb.cloud:8529") \
    .db(username="root", 
        password="RHr0KzkRUVlp61IisH8G", 
        name="DAC_devops_log",
        verify=True)

[18:14:51 -0500] [INFO]: NetworkX-cuGraph is unavailable: No module named 'cupy'.


In [3]:
arango_graph = ArangoGraph(db)
G_adb = nxadb.DiGraph(name="tasks_sprint1")

[18:17:13 -0500] [INFO]: Graph 'tasks_sprint1' exists.
[18:17:13 -0500] [INFO]: Default node type set to 'task'


In [4]:
@tool
def text_to_aql_to_text(query: str):
    """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    chain = ArangoGraphQAChain.from_llm(
    	llm=llm,
    	graph=arango_graph,
    	verbose=True,
        allow_dangerous_requests=True
    )
    
    result = chain.invoke(query)

    return str(result["result"])

In [5]:
text_to_aql_to_text('What is the most important tasks based on story points?')

/var/folders/0y/xnx3sz6927z6fg600l0yk8p40000gn/T/ipykernel_33893/4197915892.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  text_to_aql_to_text('What is the most important tasks based on story points?')




> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH task
FOR t IN task
SORT TO_NUMBER(t.StoryPoints) DESC
RETURN t

AQL Result:
[{'_key': 'TSK-B03EF789', '_id': 'task/TSK-B03EF789', '_rev': '_jTNj7BG--_', 'TaskID': 'TSK-B03EF789', 'Description': 'Perform full data migration dry run', 'StoryPoints': '13', 'Status': 'Planned', 'size': 10}, {'_key': 'TSK-2F459E7F', '_id': 'task/TSK-2F459E7F', '_rev': '_jTNjPVm--A', 'TaskID': 'TSK-2F459E7F', 'Description': 'Execute data migration cutover', 'StoryPoints': '13', 'Status': 'Planned', 'size': 10}, {'_key': 'TSK-2B1FAB57', '_id': 'task/TSK-2B1FAB57', '_rev': '_jTNjFOq--A', 'TaskID': 'TSK-2B1FAB57', 'Description': 'Implementation of payment gateway', 'StoryPoints': '13', 'Status': 'Planned', 'size': 10}, {'_key': 'TSK-B3B5A8F6', '_id': 'task/TSK-B3B5A8F6', '_rev': '_jTNjJeq---', 'TaskID': 'TSK-B3B5A8F6', 'Description': 'Implementing load balancing for the server', 'StoryPoints': '11', 'Status': 'Planned', 'size': 10}, {'_key': 'TSK-

'The most important tasks based on story points are those with the highest story points value. In this case, the tasks with the highest story points, each having a value of 13, include:\n\n1. "Perform full data migration dry run"\n2. "Execute data migration cutover"\n3. "Implementation of payment gateway"\n\nThese tasks are all currently in the "Planned" status.'

In [6]:
@tool
def text_to_nx_algorithm_to_text(query):
    """This tool is available to invoke a NetworkX Algorithm on
    the ArangoDB Graph. You are responsible for accepting the
    Natural Language Query, establishing which algorithm needs to
    be executed, executing the algorithm, and translating the results back
    to Natural Language, with respect to the original query.

    If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
    this tool.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    ######################
    print("1) Generating NetworkX code")

    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
    
    print('-'*10)
    print(text_to_nx_cleaned)
    print('-'*10)

    ######################

    print("\n2) Executing NetworkX code")
    global_vars = {"G_adb": G_adb, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        text_to_nx_final = text_to_nx
    except Exception as e:
        print(f"EXEC ERROR: {e}")
        return f"EXEC ERROR: {e}"

        # TODO: Consider experimenting with a code corrector!
        attempt = 1
        MAX_ATTEMPTS = 3

        # while attempt <= MAX_ATTEMPTS
            # ...

    print('-'*10)
    FINAL_RESULT = local_vars["FINAL_RESULT"]
    print(f"FINAL_RESULT: {FINAL_RESULT}")
    print('-'*10)

    ######################

    print("3) Formulating final answer")

    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed the following python code to help me answer my query:

        ---
        {text_to_nx_final}
        ---

        The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.

        Based on my original Query and FINAL_RESULT, generate a short and concise response to
        answer my query.
        
        Your response:
    """).content

    return FINAL_RESULT, nx_to_text


In [7]:
FINAL_RESULT, chat_answer = text_to_nx_algorithm_to_text("give me an order \
                             to do my task based on important \
                             and should be completed first if you are to use page rank? \
                             in your answer, provide task and its page rank value")

print(chat_answer)

1) Generating NetworkX code
----------
import networkx as nx

# Assuming G_adb is already defined and is a NetworkX DiGraph

# Step 1: Calculate PageRank for each task node
page_rank = nx.pagerank(G_adb)

# Step 2: Sort tasks based on PageRank value in descending order
sorted_tasks = sorted(page_rank.items(), key=lambda item: item[1], reverse=True)

# Step 3: Prepare the final result with task and its PageRank value
FINAL_RESULT = [(task, round(rank, 4)) for task, rank in sorted_tasks]
----------

2) Executing NetworkX code


[18:17:32 -0500] [INFO]: Graph 'tasks_sprint1' load took 2.560314893722534s


----------
FINAL_RESULT: [('task/TSK-5A434477', 0.0439), ('task/TSK-98C9448B', 0.0318), ('task/TSK-AE8CFDEC', 0.03), ('task/TSK-A0500206', 0.0265), ('task/TSK-FAAA4D51', 0.0234), ('task/TSK-7E9F09A9', 0.0232), ('task/TSK-1B43CB90', 0.0209), ('task/TSK-F567DBE1', 0.0204), ('task/TSK-10847035', 0.0198), ('task/TSK-42E44473', 0.0197), ('task/TSK-B03EF789', 0.0187), ('task/TSK-E6CFDD02', 0.0177), ('task/TSK-A8A24727', 0.0177), ('task/TSK-CB0C0404', 0.0177), ('task/TSK-3BA37747', 0.0168), ('task/TSK-C2D38D23', 0.0166), ('task/TSK-FBD87AD9', 0.0163), ('task/TSK-A6994DD6', 0.0156), ('task/TSK-C891B895', 0.0147), ('task/TSK-DFB54467', 0.0146), ('task/TSK-763CE1C3', 0.0139), ('task/TSK-0115FED3', 0.0136), ('task/TSK-85D5D00C', 0.0127), ('task/TSK-FAC99DF1', 0.0126), ('task/TSK-795E9201', 0.0126), ('task/TSK-E2842052', 0.0126), ('task/TSK-A0CCA417', 0.0122), ('task/TSK-29CE08AD', 0.0121), ('task/TSK-5C2187CF', 0.0119), ('task/TSK-983C30DC', 0.0117), ('task/TSK-F7E19B69', 0.0112), ('task/TSK-4276

In [17]:
FINAL_RESULT

[('task/TSK-5A434477', 0.0439),
 ('task/TSK-98C9448B', 0.0318),
 ('task/TSK-AE8CFDEC', 0.03),
 ('task/TSK-A0500206', 0.0265),
 ('task/TSK-FAAA4D51', 0.0234),
 ('task/TSK-7E9F09A9', 0.0232),
 ('task/TSK-1B43CB90', 0.0209),
 ('task/TSK-F567DBE1', 0.0204),
 ('task/TSK-10847035', 0.0198),
 ('task/TSK-42E44473', 0.0197),
 ('task/TSK-B03EF789', 0.0187),
 ('task/TSK-E6CFDD02', 0.0177),
 ('task/TSK-A8A24727', 0.0177),
 ('task/TSK-CB0C0404', 0.0177),
 ('task/TSK-3BA37747', 0.0168),
 ('task/TSK-C2D38D23', 0.0166),
 ('task/TSK-FBD87AD9', 0.0163),
 ('task/TSK-A6994DD6', 0.0156),
 ('task/TSK-C891B895', 0.0147),
 ('task/TSK-DFB54467', 0.0146),
 ('task/TSK-763CE1C3', 0.0139),
 ('task/TSK-0115FED3', 0.0136),
 ('task/TSK-85D5D00C', 0.0127),
 ('task/TSK-FAC99DF1', 0.0126),
 ('task/TSK-795E9201', 0.0126),
 ('task/TSK-E2842052', 0.0126),
 ('task/TSK-A0CCA417', 0.0122),
 ('task/TSK-29CE08AD', 0.0121),
 ('task/TSK-5C2187CF', 0.0119),
 ('task/TSK-983C30DC', 0.0117),
 ('task/TSK-F7E19B69', 0.0112),
 ('task/TS

In [8]:
pagerank_score = pd.DataFrame(FINAL_RESULT, columns=['TaskID', 'PageRank'])
pagerank_score['TaskID'] = pagerank_score['TaskID'].str.replace('task/', '')
pagerank_score = pagerank_score.sort_values(by='PageRank', ascending=False)

In [9]:
pagerank_score.head(10)

,TaskID,PageRank
0,TSK-5A434477,0.0439
1,TSK-98C9448B,0.0318
2,TSK-AE8CFDEC,0.0300
3,TSK-A0500206,0.0265
4,TSK-FAAA4D51,0.0234
5,TSK-7E9F09A9,0.0232
6,TSK-1B43CB90,0.0209
7,TSK-F567DBE1,0.0204
8,TSK-10847035,0.0198
9,TSK-42E44473,0.0197


In [ ]:
# !pip install pyvis

In [ ]:
# from pyvis.network import Network

# g = Network(height="750px", width="100%", notebook=True, directed=True, cdn_resources='remote')
# g.toggle_hide_edges_on_drag(False)
# g.barnes_hut()
# g.from_nx(G_adb)
# g.show("ex.html")

In [10]:
text_to_nx_algorithm_to_text("For each task, I need to know a value called dependency degree. \
                             This value measures the number of tasks that must be completed before this task can start. \
                             For a task, its dependecy degree is the sum of all dependency degrees of tasks that it depends on.")

1) Generating NetworkX code
----------
import networkx as nx

# Initialize a dictionary to store the dependency degree for each task
dependency_degree = {}

# Function to calculate dependency degree recursively
def calculate_dependency_degree(task):
    # If the task's dependency degree is already calculated, return it
    if task in dependency_degree:
        return dependency_degree[task]
    
    # Get all predecessors (tasks that the current task depends on)
    predecessors = list(G_adb.predecessors(task))
    
    # Base case: if there are no predecessors, the dependency degree is 0
    if not predecessors:
        dependency_degree[task] = 0
        return 0
    
    # Recursive case: sum the dependency degrees of all predecessor tasks
    total_dependency_degree = sum(calculate_dependency_degree(pred) for pred in predecessors)
    
    # Store the calculated dependency degree for the current task
    dependency_degree[task] = total_dependency_degree + len(predecessors)
    
   

"EXEC ERROR: name 'dependency_degree' is not defined"

In [18]:
# Example function to compute dependency degrees
# def compute_dependency_degrees(graph):
#     # Initialize a dictionary to store the dependency degree of each task
#     dependency_degrees = {node: None for node in graph.nodes}

#     # Recursive function to compute dependency degree with memoization
#     def calculate_dependency_degree(task):
#         # If the degree is already computed, return it
#         if dependency_degrees[task] is not None:
#             return dependency_degrees[task]

#         # If the task has no dependencies, its dependency degree is 0
#         if graph.out_degree(task) == 0:
#             dependency_degrees[task] = 0
#             return 0

#         # Otherwise, calculate the dependency degree for this task
#         total_degree = 0
#         for dependent_task in graph.predecessors(task):  # Tasks that depend on `task`
#             total_degree += 1 + calculate_dependency_degree(dependent_task)

#         dependency_degrees[task] = total_degree
#         return total_degree

#     # Perform topological sorting to ensure we process tasks in dependency order
#     for task in nx.topological_sort(graph):
#         if dependency_degrees[task] is None:
#             calculate_dependency_degree(task)

#     return dependency_degrees

import networkx as nx

# Example function to compute dependency degrees
def compute_dependency_degrees(graph):
    # Initialize a dictionary to store the dependency degree of each task
    dependency_degrees = {node: None for node in graph.nodes}

    # Recursive function to compute dependency degree with memoization
    def calculate_dependency_degree(task):
        # If the degree is already computed, return it
        if dependency_degrees[task] is not None:
            return dependency_degrees[task]

        # If the task has no dependencies, its dependency degree is 0
        if graph.out_degree(task) == 0:
            dependency_degrees[task] = 0
            return 0

        # Otherwise, calculate the dependency degree for this task
        total_degree = 0
        for dependent_task in graph.predecessors(task):  # Tasks that depend on `task`
            total_degree += 1 + calculate_dependency_degree(dependent_task)

        dependency_degrees[task] = total_degree
        return total_degree

    # Perform topological sorting to ensure we process tasks in dependency order
    for task in nx.topological_sort(graph):
        if dependency_degrees[task] is None:
            calculate_dependency_degree(task)

    # Convert the dependency degrees dictionary to a list of tuples (task, degree)
    dependency_degree_list = [(task, degree) for task, degree in dependency_degrees.items()]

    return dependency_degree_list

dependency_degree = compute_dependency_degrees(G_adb)


[20:10:15 -0500] [INFO]: Graph 'tasks_sprint1' load took 22.80359387397766s


In [19]:
dependency_degree

[('task/TSK-10847035', 0),
 ('task/TSK-F7E19B69', 7),
 ('task/TSK-5C2187CF', 17),
 ('task/TSK-FBD87AD9', 17),
 ('task/TSK-4A3587C9', 1),
 ('task/TSK-977CB080', 1),
 ('task/TSK-A0500206', 65),
 ('task/TSK-89806FDF', 0),
 ('task/TSK-FBED5511', 0),
 ('task/TSK-7E9F09A9', 39),
 ('task/TSK-30DBEA24', 0),
 ('task/TSK-2179FF75', 0),
 ('task/TSK-514F6016', 0),
 ('task/TSK-5A434477', 0),
 ('task/TSK-B03EF789', 19),
 ('task/TSK-89BBB564', 6),
 ('task/TSK-59CE546C', 4),
 ('task/TSK-2970AC03', 0),
 ('task/TSK-CAFF3B09', 1),
 ('task/TSK-0885F3E9', 0),
 ('task/TSK-1B43CB90', 0),
 ('task/TSK-FB3E490E', 3),
 ('task/TSK-EBA1C369', 0),
 ('task/TSK-E2E87C0B', 0),
 ('task/TSK-763CE1C3', 11),
 ('task/TSK-2F459E7F', 0),
 ('task/TSK-C2D38D23', 13),
 ('task/TSK-89754CBF', 1),
 ('task/TSK-795E9201', 0),
 ('task/TSK-FD3B1EFF', 2),
 ('task/TSK-AE8CFDEC', 0),
 ('task/TSK-46138F7D', 0),
 ('task/TSK-BE4188D4', 0),
 ('task/TSK-DA566BCB', 0),
 ('task/TSK-C891B895', 0),
 ('task/TSK-1FE396B7', 4),
 ('task/TSK-39885F7F'

In [20]:
dependency_score = pd.DataFrame(dependency_degree, columns=['TaskID', 'Dependency'])
dependency_score['TaskID'] = dependency_score['TaskID'].str.replace('task/', '')
dependency_score = dependency_score.sort_values(by='Dependency', ascending=False)
dependency_score.head(10)

,TaskID,Dependency
6,TSK-A0500206,65
9,TSK-7E9F09A9,39
14,TSK-B03EF789,19
2,TSK-5C2187CF,17
3,TSK-FBD87AD9,17
41,TSK-DFB54467,16
59,TSK-983C30DC,13
26,TSK-C2D38D23,13
24,TSK-763CE1C3,11
1,TSK-F7E19B69,7


In [25]:
# Merge the two dataframes and sort by both Dependency and PageRank
merged_df = pd.merge(pagerank_score, dependency_score, on='TaskID')
merged_df = merged_df.sort_values(by=['Dependency', 'PageRank'], ascending=[True, False])
merged_df

,TaskID,PageRank,Dependency
0,TSK-5A434477,0.0439,0
1,TSK-98C9448B,0.0318,0
2,TSK-AE8CFDEC,0.0300,0
4,TSK-FAAA4D51,0.0234,0
6,TSK-1B43CB90,0.0209,0
...,...,...,...
16,TSK-FBD87AD9,0.0163,17
28,TSK-5C2187CF,0.0119,17
10,TSK-B03EF789,0.0187,19
5,TSK-7E9F09A9,0.0232,39
